# Clustering using keywords

The first idea was to use the predefined keywords in the metadata records, to build a list of different concepts and to group the results based on these.

The first step is to build a keyword list that can be used for text classifcation. In order to do this, we get the list of keywords from the metadata records and select the ones that are the similar to the search keyword (above a predefined threshold). Also, the keywords must be lemmatized in order to avoid differences between singular, plural word or case-sensitivity.

For each metadata record, we are looking at the keywords that are attached and, if they are also contained in the list of the best keywords, then the preprocessed metatdata record is written in the training file, together with its corresponding best keywords, as labels. The same text will be written also in the test file, so the classifier will be able to find the best category for each metadata record.

In this case, the classifier used is fasttext[1] and it uses a pre-trained model for English word vectors.

[1] https://fasttext.cc/

In [ ]:
#The first method is based on the keywords metadata and
#word similarity

%run "NLP_clustering.ipynb"

if __name__ == "__main__":

    csw = CatalogueServiceWeb('http://geocatalog.webservice-energy.org/geonetwork/srv/eng/csw')
    
    set_title = fes.PropertyIsLike('any', '')
    filter_list = [set_title]

    preprocessed_list_of_titles = []
    csw.getrecords2(constraints=filter_list, maxrecords=2000)

    fmt = '{:*^64}'.format
    print(fmt(' Catalog information '))
    print("CSW version: {}".format(csw.version))
    print("Number of datasets available: {}".format(len(csw.records.keys())))
    print('\n')

    keywords_set = []

    for rec in csw.records:
        keywords_set = build_keywords_set(csw.records[rec].subjects, keywords_set)
        title = csw.records[rec].title
        
        if csw.records[rec].abstract != None:
            title = title + " " + csw.records[rec].abstract

    model, index2word_set = init_language_model()

    sent1 = avg_feature_vector(SEARCH_QUERY, model, num_features=NUM_FEATURES, index2word_set=index2word_set)
    similarity_index = 0.5
    count = 0
    while similarity_index < 1:
        best_keywords = get_best_keywords_from_list(keywords_set, sent1, similarity_index)
        build_training_test_database(csw.records, best_keywords)
        train_fasttext_classifer()
        run_classifier_on_data(count)
        show_results(str(similarity_index), count)
        similarity_index += 0.1
        count+= 1
        

In [ ]:
show_results("0.8", 3)